In [4]:
library(DBI)
library(RSQLite)

In [2]:
conn = dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")
# 데이터 베이스를 하나 만들고 싶을 때,
# DBMS는 SQLite()를 이용
# 어디에? c:/RSQL 폴더 아래에 Customer.sqlite라는 이름으로

# dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")라는 connector를 conn이라고 저장해준 것

- 만들어 놓기만 한 거고 내용은 아무것도 없다.

In [15]:
# 데이터 베이스와의 연결 해제
# dbDisconnect(conn)

In [16]:
##### PRAGMA sta dbDisconnect(conn)tement : SQLite에 특정된 SQL 확장

###  sqlite에서 외래 키를 설정하기 위해서는 다음 문장을 실행해야 함
dbExecute(conn, "PRAGMA foreign_keys=ON")

# Customer_Info 테이블 생성하기
sql = "CREATE TABLE IF NOT EXISTS Customer_Info
					( 	CID TEXT PRIMARY KEY, 
						Name TEXT,
						Address TEXT,
						Tel TEXT,
						BDay TEXT,
						SID TEXT,
						Job TEXT,
						Sex TEXT,
						Married TEXT )"
#   ( 	CID TEXT PRIMARY KEY, 
# 						Name TEXT,
# 						Address TEXT,
# 						Tel TEXT,
# 						BDay TEXT,
# 						SID TEXT,
# 						Job TEXT,
# 						Sex TEXT,
# 						Married TEXT )

# CREATE TABLE IF NOT EXISTS Customer_Info 
# -> Customer_Info라는 TABLE이 없으면 만들어라,있으면 만들지 말고
# 이런 이름(CID, Name, Address, Tel, BDay, SID, ...)들의 필드를 가진, type은 TEXT인.

# 위에서 정의했던 conn이라는 connector이용해서 sqp라는 문장을 실행
rs = dbSendQuery(conn, sql)
# dbSendQuerry라는 함수를 불러서 반환되는 값을
# 굳이 rs라는 변수에 저장하고
# 왜 아래에서 또 rs를 굳이 제거?
# 사실 dbSendQuery(conn, sql) 이것만 실행해도 되긴 함
# 매뉴얼에서 dbSendQuery라는 함수를 보면
# dbSendQuery라는 함수는 S4 object를 반환하는데,,
# 만약 이 반환되는 값을 임의의 변수에 저장하지 않으면
# error는 아니고 warning이 출력됨
# 왜?
# dbSendQuery가 안 닫혀서..
# 그냥 예전에 최귭 교수님이 파일 열었으면 닫아야된다고 했던 거랑
# 같은 원리인 것 같음
# 이것도 dbSendQuery사용했으면 제거해주는 게 그냥 .. 깔끔하다고 생각하면 될 듯

# rs 제거
dbClearResult(rs)

# 테이블이 생성되었는지(존재하는지) 확인
dbExistsTable(conn, "Customer_Info")

[1] 0

[1] TRUE

- TRUE라고 반환됨
  - 만들어졌음을 알 수 있음
  - 내용은 없고 table만 생성된 것임
  - table안에 data 넣고 빼고 혹은 추가 table 생성 등의 작업을 할 수 있음

In [17]:
sql = "CREATE TABLE IF NOT EXISTS Customer_Score  
				( 	CID TEXT, 
					Year TEXT not null,
					On_Score INTEGER default 0,
					Off_Score INTEGER default 0,
					PRIMARY KEY (CID, Year),
					FOREIGN KEY(CID) REFERENCES Customer_Info(CID) )"

rs = dbSendQuery(conn, sql)

# rs 제거
dbClearResult(rs)

# 테이블이 생성되었는지(존재하는지) 확인
dbExistsTable(conn, "Customer_Score")

[1] TRUE

- 2개의 table을 만듦

In [18]:
dbListTables(conn)

[1] "Customer_Info"  "Customer_Score"

In [19]:
# # Customer_Score 테이블 삭제하기
# rs = dbSendQuery(conn, "DROP TABLE Customer_Score")

# # rs 제거
# dbClearResult(rs)

# # 테이블이 존재하는지(삭제되었는지) 확인
# dbExistsTable(conn, "Customer_Score")

---

> 데이터 삽입, 갱신, 삭제

In [20]:
# Customer_Info 테이블에 데이터 insert
sql = " INSERT INTO Customer_Info 
VALUES ('100001', '김민지', '서울시 광진구', '423-8627', 
'1975-07-01', '750701-2534218', '공무원', '1', '0') "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

ERROR: Error: UNIQUE constraint failed: Customer_Info.CID


In [21]:
# Customer_Info 테이블에 데이터 삽입 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Info")
# * => 해당 table에서 모든 field를 불러와라

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0


In [22]:
sql = " INSERT INTO Customer_Score 
VALUES ('100001', '2012', 42, 111) "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

sql = " INSERT INTO Customer_Score 
VALUES ('100001', '2013', 58, 89) "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 데이터 삽입 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,89


- Customer_Info 테이블에 있는 데이터 update

In [25]:
sql = "UPDATE Customer_Info SET Address = '서울시 영등포구' 
					WHERE CID = '100001' "
# CID = '100001'을 만족하는 고객만 update한다.

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [26]:
dbGetQuery(conn, "SELECT * FROM Customer_Info")

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 영등포구,423-8627,1975-07-01,750701-2534218,공무원,1,0


---

In [28]:
# Customer_Score 테이블에 있는 데이터 update
sql = "UPDATE Customer_Score SET Off_Score = 98 
WHERE CID = '100001' and Year='2013' "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 데이터 갱신 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,98


In [29]:
# Customer_Score 테이블에 있는 특정 데이터 delete
sql = "DELETE FROM Customer_Score 
WHERE CID = '100001' and Year='2013' "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 특정 데이터 삭제 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111


In [30]:
# Customer_Score 테이블에 있는 전체 데이터 delete
sql = "DELETE FROM Customer_Score "
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 전체 데이터 삭제 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>


---

> dbWriteTable()를 이용하여 데이터를 한 번에 삽입해보자

In [32]:
# Customer 데이터베이스 연결
conn = dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")

# Customer 데이터베이스 안에 있는 테이블 리스트 확인하기
dbListTables(conn)

# CSV 화일의 데이터를 Customer_Info 테이블에 삽입하기
customer_data = read.csv('customer_data.csv')
dbWriteTable(conn = conn, name = "Customer_Info", customer_data, overwrite=T, row.names=FALSE)

# Customer_Info 테이블에 있는 데이터 확인하기
dbReadTable(conn, "Customer_Info")

[1] "Customer_Info"  "Customer_Score"

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


- 혹은 아래와 같이 확인할 수 있다. 

In [35]:
dbGetQuery(conn, "SELECT * FROM Customer_Info ")
# 귀찮으니 dbReadTable로 처리하자

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [36]:
customer_score = read.csv('customer_score.csv')
dbWriteTable(conn = conn, name = "Customer_Score", customer_score,overwrite=T, row.names=FALSE)

# Customer_Score 테이블에 있는 데이터 확인하기
dbReadTable(conn, "Customer_Score")

CID,Year,On_Score,Off_Score
<int>,<int>,<int>,<int>
100001,2012,42,111
100002,2012,12,30
100003,2012,86,128
100004,2012,0,87
100005,2012,122,56
100006,2012,98,17
100007,2012,0,0
100008,2012,56,172
100009,2012,24,45


----

In [1]:
library(DBI)
library(RSQLite)

In [2]:
conn = dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")

In [3]:
sql = "SELECT CID, Name, Job FROM Customer_Info"
dbGetQuery(conn, sql)

CID,Name,Job
<int>,<chr>,<chr>
100001,김민주,공무원
100002,정영미,회사원
100003,최소영,학생
100004,박진재,공무원
100005,이순석,회사원
100006,박선희,회사원
100007,김화연,학생
100008,박민선,학생
100009,노상수,공무원


- 전체 field 가져오기

In [4]:
sql = "SELECT all * FROM Customer_Info"
dbGetQuery(conn, sql)
# 또는
sql = "SELECT * from Customer_Info"
dbGetQuery(conn, sql)
# 또는
sql = "SELECT CID, Name, Address, Tel, Bday, SID, Job, Sex, Married FROM Customer_Info"
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


- 검색 결과에서 레코드의 중복 제거

In [5]:
# 테이블 Customer_Info에는 Job 필드가 존재하며, 고객들의 직업은 중복해서 나타난다. 중복을 제거하고 고객들의 직업을 검색해 보아라.

sql <- "SELECT DISTINCT Job,sex FROM Customer_Info"
dbGetQuery(conn, sql)

# 따로따로가 아니고 조합해서 처리함

Job,Sex
<chr>,<int>
공무원,1
회사원,1
학생,1
공무원,0
회사원,0
주부,1


- 특정한 레코드의 조건 검색 : WHERE

In [6]:
# Customer_Info 테이블에서 직업이 ‘회사원’인 고객의 고객번호(CID), 이름(Name), 직업(Job), 결혼여부(Married)를 검색해 보아라.

sql = "SELECT  CID, Name, Job, Married 
FROM  Customer_Info
WHERE  Job = '회사원' "
dbGetQuery(conn, sql)

CID,Name,Job,Married
<int>,<chr>,<chr>,<int>
100002,정영미,회사원,1
100005,이순석,회사원,0
100006,박선희,회사원,0
100010,이민석,회사원,1


In [7]:
# Customer_Info 테이블에서 직업이 ‘회사원’이 아닌 고객의 고객번호(CID), 이름(Name), 직업(Job), 결혼여부(Married)를 검색해 보아라.

sql = "SELECT  CID, Name, Job, Married 
FROM  Customer_Info
WHERE  Job != '회사원' "
dbGetQuery(conn, sql)

CID,Name,Job,Married
<int>,<chr>,<chr>,<int>
100001,김민주,공무원,0
100003,최소영,학생,0
100004,박진재,공무원,1
100007,김화연,학생,0
100008,박민선,학생,0
100009,노상수,공무원,1
100011,정영미,주부,1


In [8]:
# Customer_Info 테이블에서 성별(Sex)과 결혼여부(Married)가 모두 1인 고객의 고객번호(CID), 이름(Name), 성별(Sex), 결혼여부(Married)를 
#검색해 보아라.
sql = "SELECT  CID, Name, Sex, Married
FROM  Customer_Info
WHERE  Sex='1' and Married='1' "
dbGetQuery(conn, sql)

CID,Name,Sex,Married
<int>,<chr>,<int>,<int>
100002,정영미,1,1
100011,정영미,1,1


In [9]:
# Customer_Score 테이블에서 2013년의 온라인점수(On_Score)가 50 이상이고 100이하인 레코드를 검색해 보아라.
sql = "SELECT CID, Year, On_Score
FROM Customer_Score
WHERE Year=2013 and 
On_Score>=50 and On_Score<=100 "
dbGetQuery(conn, sql)

CID,Year,On_Score
<int>,<int>,<int>
100001,2013,58
100002,2013,98
100006,2013,66


In [10]:
sql = "SELECT * from Customer_Score"
dbGetQuery(conn, sql)

CID,Year,On_Score,Off_Score
<int>,<int>,<int>,<int>
100001,2012,42,111
100002,2012,12,30
100003,2012,86,128
100004,2012,0,87
100005,2012,122,56
100006,2012,98,17
100007,2012,0,0
100008,2012,56,172
100009,2012,24,45


In [11]:
# Customer_Score 테이블에는 온라인점수(On_Score)와 오프라인점수(Off_Score)가 년도별로 집계되어 있다. 
# 2013년의 온라인 고객점수와 오프라인 고객점수의 합(TotalScore)을 검색해 보아라.
sql = "SELECT CID, Year, On_Score+Off_Score TotalScore
FROM Customer_Score
WHERE Year=2013 "
dbGetQuery(conn, sql)

CID,Year,TotalScore
<int>,<int>,<int>
100001,2013,147
100002,2013,138
100003,2013,530
100004,2013,74
100005,2013,488
100006,2013,78
100007,2013,5
100008,2013,142
100009,2013,8


In [12]:
# Customer_Score 테이블에서 2013년의 온라인 고객점수와 오프라인 고객점수의 합(TotalScore)이 100 이상인 고객을 검색해 보아라.

sql = "SELECT CID, Year, On_Score+Off_Score TotalScore
FROM Customer_Score
WHERE Year=2013 and TotalScore>100 "
dbGetQuery(conn, sql)


CID,Year,TotalScore
<int>,<int>,<int>
100001,2013,147
100002,2013,138
100003,2013,530
100005,2013,488
100008,2013,142


In [13]:
# Customer_Info 테이블에서 주소(Address)가 널 값이 아닌 레코드를 검색해 보아라.

sql = "SELECT *
FROM Customer_Info
WHERE Address IS NOT NULL "
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [14]:
# Customer_Info 테이블에서 생년월일이 1970년 1월 1일부터 1970년 12월 31일 사이에 있는 고객들을 검색해 보아라.

sql = "SELECT CID, Name, BDay, Job
FROM Customer_Info
WHERE BDay between '1970-01-01' and '1979-12-31' "
dbGetQuery(conn, sql)

CID,Name,Bday,Job
<int>,<chr>,<chr>,<chr>
100001,김민주,1975-07-01,공무원
100002,정영미,1977-02-01,회사원
100006,박선희,1975-04-05,회사원
100010,이민석,1972-08-02,회사원
100011,정영미,1974-09-21,주부


In [15]:
# Customer_Score 테이블에서 2013년의 온라인 고객점수와 오프라인 고객점수의 합(TotalScore)이 
# 100 이상, 500이하인 고객을 검색해 보아라.

sql = "SELECT CID, Year, On_Score+Off_Score TotalScore
FROM Customer_Score
WHERE Year=2013 and 
TotalScore BETWEEN 100 AND 500 "
dbGetQuery(conn, sql)

CID,Year,TotalScore
<int>,<int>,<int>
100001,2013,147
100002,2013,138
100005,2013,488
100008,2013,142


In [16]:
# Customer_Info 테이블에서 ‘김’씨 성을 가진 고객의 고객번호, 이름, 주소를 검색해 보아라.

sql = "SELECT CID, Name, Address
FROM Customer_Info
WHERE Name LIKE '김%' "
dbGetQuery(conn, sql)


CID,Name,Address
<int>,<chr>,<chr>
100001,김민주,서울시 광진구
100007,김화연,경남 경주시


In [17]:
# Customer_Info 테이블에서 주소가 제주도 ‘서귀포’인 고객을 검색해 보아라.

sql = "SELECT *
FROM Customer_Info
WHERE Address LIKE '%서귀포%' "
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0


In [18]:
# Customer_Info 테이블에서 직업이 ‘회사원’, ‘공무원’인 고객을 검색해 보아라.

sql = "SELECT CID, Name, Job
FROM Customer_Info
WHERE Job IN ('회사원', '공무원') "
dbGetQuery(conn, sql)

CID,Name,Job
<int>,<chr>,<chr>
100001,김민주,공무원
100002,정영미,회사원
100004,박진재,공무원
100005,이순석,회사원
100006,박선희,회사원
100009,노상수,공무원
100010,이민석,회사원


- 데이터의 그룹화 GROUP BY

In [19]:
# Customer_Info 테이블에서 직업별 고객 빈도를 파악해 보아라. 

sql = "SELECT Job, COUNT(*) freq
FROM Customer_Info
GROUP BY Job "
dbGetQuery(conn, sql)

Job,freq
<chr>,<int>
공무원,3
주부,1
학생,3
회사원,4


In [20]:
# Customer_Score 테이블에는 고객들의 2012년과 2013년 점수가 들어있다. 
# On_Score가 모두 0보다 큰 고객들의 On_Score의 합을 구해 보아라.

sql = "SELECT CID, sum(On_Score) 
FROM Customer_Score
WHERE On_Score > 0 
GROUP BY CID "
dbGetQuery(conn, sql)

CID,sum(On_Score)
<int>,<int>
100001,100
100002,110
100003,488
100005,478
100006,164
100008,100
100009,26


In [21]:
# Customer_Score 테이블에서 2012년과 2013년 자료의 고객별 Off_Score 평균 점수를 구해 보아라.

sql = "SELECT CID, avg(Off_Score) 
FROM Customer_Score
GROUP BY CID "
dbGetQuery(conn, sql)

CID,avg(Off_Score)
<int>,<dbl>
100001,100.0
100002,35.0
100003,128.0
100004,80.5
100005,94.0
100006,14.5
100007,2.5
100008,135.0
100009,25.5


- 데이터 그룹화의 조건 검색 : HAVING
  - GROUP BY 절에서 명시된 그룹에 대한 검색 조건을 주고자 할 때 HAVING 절을 이용
  - GROUP BY에 대한 조건이라고 생각하면 됨

In [22]:
# Customer_Info 테이블에서 직업별 고객 빈도가 3이상인 직업을 파악해 보아라. 

sql = "SELECT Job, COUNT(*) freq
FROM Customer_Info
GROUP BY Job 
HAVING freq >= 3 "
dbGetQuery(conn, sql)

Job,freq
<chr>,<int>
공무원,3
학생,3
회사원,4


In [24]:
# Customer_Score 테이블에는 고객들의 2012년과 2013년 점수가 들어있다. 
# 2012년과 2013년 On_Score의 합이 100보다 큰 고객들의 명단을 구해 보아라.

sql = "SELECT CID, sum(On_Score) totalScore
FROM Customer_Score
GROUP BY CID 
HAVING totalScore > 100 "
dbGetQuery(conn, sql)

CID,totalScore
<int>,<int>
100002,110
100003,488
100005,478
100006,164


- 출력 순서를 명시하는 검색 : ORDER BY

In [25]:
# Customer_Info 테이블에서 고객들의 직업을 정렬하여 출력해 보아라.
sql = "SELECT * 
FROM Customer_Info
ORDER BY Job "
dbGetQuery(conn, sql)


CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1
100011,정영미,서울시 관악구,272-6745,1974-09-21,740921-2782216,주부,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0


In [26]:
# Customer_Score 테이블에서 2012년과 2013년 자료의 고객별 Off_Score 평균이 50보다 큰 점수들을 내림차순으로 출력해 보아라.

sql = "SELECT CID, avg(Off_Score) mean
FROM Customer_Score
GROUP BY CID 
HAVING mean > 50 
ORDER BY mean "
dbGetQuery(conn, sql)


CID,mean
<int>,<dbl>
100004,80.5
100005,94.0
100001,100.0
100003,128.0
100008,135.0


---

> 출력 순서들 명시하는 검색 :  ORDER BY

    질의 결과로 나오는 레코드들을 정렬시키고자 할 때 ORDER BY 절을 이용한다.
    ORDER BY 절에서 레코드를 정렬할 때 기준을 삼고자 하는 필드 이름을 지정하면 기본적으로 해당 필드의 값에 따라 레코드가 오름차순으로 정렬된다.

In [13]:
# Customer_Info 테이블에서 고객들의 직업을 정렬하여 출력
sql = "SELECT * FROM Customer_Info ORDER BY Job"
dbGetQuery(conn, sql)
# default는 ASC, 따라서 써도 그만 안 써도 그만
# 내림차순으로 정렬하려면 DESC

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1
100011,정영미,서울시 관악구,272-6745,1974-09-21,740921-2782216,주부,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0


In [14]:
# Customer_Score 테이블에서 2012년과 2013년 자료의 고객별 Off_Score 평균이 
# 50보다 큰 점수들을 내림차순으로 출력해 보아라.
sql = "SELECT CID, avg(Off_Score) mean
FROM Customer_Score
GROUP BY CID 
HAVING mean > 50 
ORDER BY mean DESC"
dbGetQuery(conn, sql)

CID,mean
<int>,<dbl>
100008,135.0
100003,128.0
100001,100.0
100005,94.0
100004,80.5


---

> 둘 이상의 테이블에서 검색 : JOIN

     둘 이상의 테이블에서 원하는 데이터를 검색하고자 할 때는 조인(JOIN) 문장을 이용한다.
    

In [15]:
# 테이블 Customer_Info와 Customer_Score에서 
# 고객번호(CID)와 이름(Name), 2013년 온라인 점수(On_Score)를 고객번호 순으로 검색해 보아라.

sql = "SELECT C.CID, C.Name, CS.On_Score 
FROM Customer_Info C
JOIN Customer_Score CS 
ON C.CID = CS.CID
WHERE CS.Year=2013 "
dbGetQuery(conn, sql)

# C라는 이름으로 Customer_Info를 사용하겠다.
# CS라는 이름으로 Customer_Score를 사용하겠다.

# FROM Customer_Info C 이 TABLE을 기준으로 하여
# JOIN Customer_Score CS을 합쳐줌

# ON의 의미? 
# JOIN할 때 기준이 되어줌

CID,Name,On_Score
<int>,<chr>,<int>
100001,김민주,58
100002,정영미,98
100003,최소영,402
100004,박진재,0
100005,이순석,356
100006,박선희,66
100007,김화연,0
100008,박민선,44
100009,노상수,2


In [21]:
# 테이블 Customer_Score와 Customer_Info에서 고객번호(CID), 이름(Name), 2012년과 2013년 자료의 고객별 온라인 점수(On_Score)와 오프라인 점수(Off_Score)의 합을 점수의 합 순으로 검색해 보아라.

sql = "SELECT CS.CID, C.Name, sum(On_Score+Off_Score) total 
FROM Customer_Score CS
JOIN Customer_Info C 
ON CS.CID = C.CID
GROUP BY CS.CID, C.Name
ORDER BY total DESC "
dbGetQuery(conn, sql)
# 두개 JOIN말고 3개 ,4개 하고 싶으면 ON CS.CID = C.CID
# 이 밑에다가
# JOIN Customer_OnlineInfo co
# ON CS.CID = CO.CID
# 이렇게 입력하여 3번째 TABLE을 JOIN할 수 있다. 

# ON CS.CID = CO.CID 이것만 잘 입력하면 된다
# 입력 순서는 지켜줘야하는 것 같다.

# 그 후에 ORDER BY나 HAVING이나 GROUP BY 사용 가능함

CID,Name,total
<int>,<chr>,<int>
100003,최소영,744
100005,이순석,666
100008,박민선,370
100001,김민주,300
100006,박선희,193
100002,정영미,180
100004,박진재,161
100011,정영미,87
100009,노상수,77


> ### INNER JOIN, OUTER JOIN 알아보기 (필요하면...)

In [18]:
# 테이블 Customer_Score와 Customer_Info에서 고객번호(CID), 이름(Name), 2012년과 2013년 자료의 고객별 온라인 점수(On_Score)와 오프라인 점수(Off_Score)의 합을 점수의 합 순으로 검색해 보아라.

sql = "SELECT CS.CID, C.Name 
FROM Customer_Score CS
LEFT OUTER JOIN Customer_Info C 
ON CS.CID = C.CID
"
dbGetQuery(conn, sql)

CID,Name
<int>,<chr>
100001,김민주
100002,정영미
100003,최소영
100004,박진재
100005,이순석
100006,박선희
100007,김화연
100008,박민선
100009,노상수


> 중첩 질의 : IN

    일부 질의들은 데이터베이스 내에 존재하는 값들을 검색한 다음 이를 비교 조건에서 사용한다.
    이러한 질의들은 중첩 질의를 사용해서 편리하게 나타낼 수가 있다.
    중첩 질의는 어떤 질의의 WHERE 문에 완전한 하나의 SELECT 질의가 포함되어 있는 형태의 질의이다.
    
    하나의 sql문장 안에 두개의 sql문장 중첨 시키는 형식
    if문 중첩과 유사
    
    중첩된 괄호 안부터 확인한다.

In [23]:
# Customer_Score 테이블에 있는 고객들 중에서 2013년 On_Score가 0보다 큰 고객의 명단을 Customer_Info에서 검색해 보아라.

sql = "SELECT CID, Name
FROM Customer_Info
WHERE CID IN ( 	SELECT CID
FROM Customer_Score
WHERE Year=2013 and On_Score > 0 )  "
dbGetQuery(conn, sql)

CID,Name
<int>,<chr>
100001,김민주
100002,정영미
100003,최소영
100005,이순석
100006,박선희
100008,박민선
100009,노상수


In [25]:
# Customer_Info 테이블에 있는 고객들 중에서 Customer_Score 테이블의 2013년 On_Score가 0보다 큰 고객번호와 점수를 검색해 보아라.

sql = "SELECT CID, On_Score
FROM Customer_Score
WHERE CID IN ( 	SELECT CID
FROM Customer_Info  ) 
and Year=2013 and On_Score > 0  "
dbGetQuery(conn, sql)

CID,On_Score
<int>,<int>
100001,58
100002,98
100003,402
100005,356
100006,66
100008,44
100009,2
